# Solución de problemas con regresión Ridge

En este tutorial conocerás cómo crear un modelo de regresión con la técnica de regularización con norma $L_2$, también llamada regresión Ridge. Para encontrar el mejor valor de su hiperparámetro, realizaremos un ajuste mediante la técnica de validación cruzada con k-Folds. En ese sentido, veremos cómo realizar los siguientes procesos:

1. Importar las librerías necesarias.
2. Cargar un conjunto de datos.
3. Preparar los datos para el modelado.
4. Entrenar un modelo de referencia usando regresión lineal.
5. Realizar una búsqueda de hiperparámetros.

Utilizaremos el conjunto de datos correspondiente a la caracterización de casas y su precio. Nuestro objetivo es, entonces, obtener un modelo para predecir el precio de una vivienda dadas sus características.

## 1. Importación de librerías requeridas

Importaremos la librería `pandas` y `scikit-learn`. En particular, usaremos las siguientes clases para entrenar el modelo de regresión Ridge:

* `GridSearchCV`: clase para entrenar múltiples modelos variando sus parámetros. Se utiliza para hacer una búsqueda exhaustiva de los mejores valores para el entrenamiento de un modelo.
* `KFold`: clase para definir múltiples conjuntos de entrenamiento y validación sobre el conjunto de datos.
* `MinMaxScaler`, `RobustScaler` y `StandardScaler`: clases para escalar los valores de nuestro conjunto de datos.
* `Ridge`: clase para crear y entrenar un modelo de regresión regularizada con norma $L_2$.

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

from importlib.metadata import version

print(f"Versión de Pandas: {version('pandas')}")
print(f"Versión de Scikit-learn: {version('scikit-learn')}")

## 2. Carga de datos

Realizaremos la carga de datos usando la función de Pandas `read_csv()`, especificando la ruta y el separador del archivo:

In [ ]:
data_raw = pd.read_csv('data/kc_house_data.csv', sep=',')

Veremos los primeros datos del conjunto usando `head()`:

In [ ]:
data_raw.head()

## 3. Preparación de los datos

Definiremos la variable `data` para almacenar un conjunto de datos modificado. En ese sentido, podremos tener una copia de los datos originales almacenada en la variable `data_raw`, por si en algún momento es necesario recuperar información que haya sido modificada:

In [ ]:
data = data_raw.copy()

### Eliminación de variables poco relevantes

Eliminaremos tres variables con poca relevancia para el precio de las viviendas: `id`, `date` y `zipcode`, haciendo uso de la función `drop()` sobre nuestro DataFrame:

In [ ]:
data = data.drop(['id','date','zipcode'], axis="columns")

Veremos el resultado con `data.head()`:

In [ ]:
data.head()

### División de datos

Ahora dividiremos el conjunto de datos resultante en un conjunto de entrenamiento y uno de pruebas mediante la función `train_test_split()`. Usaremos el 80% de los datos para el entrenamiento y el 20% restante para las pruebas:

In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=9)
train.head()

Como los algoritmos supervisados implementados en `scikit-learn` necesitan que las variables de entrada estén separadas de la variable objetivo, usaremos la función `drop` y definiremos `x_train` y `y_train`, que representan los valores de las variables independientes y los valores de la variable objetivo, respectivamente:

In [ ]:
x_train = train.drop(['price'],axis="columns")
y_train = train['price']

### Estandarización

Para el caso de regresión regularizada, la escala en la que se encuentran las variables se vuelve relevante, a diferencia de la regresión lineal simple. Específicamente, dado que los métodos de regularización actúan sobre la magnitud de los coeficientes del modelo, todos deben estar en la misma escala. En este caso utilizaremos un objeto de la clase `MinMaxScaler()`.

En la variable `scaler` asignamos el objeto que corresponde al escalador para las variables numéricas, en Scikit-learn hay varios y los más comunes son los siguientes:

* **StandardScaler**: Escala las variables para que la distribución de probabilidad de cada variable se comporte como una distribución normal (es decir, media 0 y varianza 1)
* **MinMaxScaler**: Escala las variables en un rango especificado (que es [0,1] por defecto y puede definirse manualmente).
* **RobustScaler**: Remueve la media y escala las variables usando el rango interquartil. Este es útil cuando las variables tienen valores atípicos (_outliers_)

#### Experimento

Intenta cambiar el escalador de la celda siguiente usando `StandardScaler()` o `RobustScaler()` y revisa qué efectos tiene cada uno en los coeficientes resultantes en la regresión y en el rendimiento del modelo. Diferentes escaladores tendrán mejores o peores resultados dependiendo de los datos.

Si deseas ir más allá, puedes agregar el escalado de variables a la fase de búsqueda de hiperparámetros.

In [ ]:
scaler = MinMaxScaler()
scaler

In [ ]:
columns = x_train.columns
x_train = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train, columns=columns)

Visualizaremos los nuevos valores del conjunto de entrenamiento usando `head()`:

In [ ]:
x_train.head()

## 4. Entrenamiento de un modelo de referencia

Con el conjunto de datos modificado, ahora entrenaremos un modelo de referencia, que nos permitirá ver cómo es el rendimiento de una regresión lineal simple con este conjunto de datos. Definiremos un objeto de tipo `LinearRegression()` y lo entrenaremos con la función `fit()`, utilizando el conjunto de entrenamiento separado en las variables independientes `x_train` y la variable objetivo `y_train`:

In [ ]:
reg_lineal = LinearRegression().fit(x_train, y_train)

Ahora veremos los coeficientes y el intercepto resultantes:

In [ ]:
print ('Coeficientes: ', reg_lineal.coef_)
print ('Intercepto: ', reg_lineal.intercept_)

Específicamente, cada variable tiene los siguientes coeficientes, que obtendremos usando `reg_lineal.coef_`:

In [ ]:
pd.DataFrame(zip(x_train.columns, reg_lineal.coef_),columns=["Variable","Coeficiente"])

### Evaluación del modelo

Ahora utilizaremos el conjunto de pruebas para evaluar el desempeño del modelo. Separaremos las variables independientes y la variable objetivo, de la misma forma que para el conjunto de entrenamiento:

In [ ]:
x_test = test.drop(['price'],axis="columns")
y_test = test['price']

También usaremos la variable `scaler` para escalar las variables independientes del conjunto de pruebas. Utilizaremos solamente la información del conjunto de entrenamiento, con el método `transform()`:

In [ ]:
x_test = scaler.transform(x_test)
x_test = pd.DataFrame(x_test, columns=columns)

Finalmente, realizaremos las predicciones. Utilizaremos tres métricas para evaluar el desempeño del modelo de referencia: la raíz del error cuadrático medio, el error absoluto medio y el coeficiente de determinación: 

In [ ]:
y_pred = reg_lineal.predict(x_test)

print('------ Modelo de regresión lineal simple----')
print("RMSE: %.2f" % root_mean_squared_error(y_test, y_pred))
print("MAE: %.2f" % mean_absolute_error(y_test, y_pred))
print('R²: %.2f' % r2_score(y_test, y_pred))

Como puedes observar, el modelo de regresión lineal simple se ajusta un 71% a los datos. En este caso, se tienen errores medios en el orden de los cientos de miles de dólares. Ahora veremos cómo se compara un modelo de regresión Ridge.

## 5. Búsqueda de hiperparámetros

Cambiar el valor de los hiperparámetros tiene un impacto directo sobre el desempeño del modelo resultante, por lo que queremos encontrar un valor que resulte en el mejor desempeño posible. A continuación, ejecutaremos un procedimiento de búsqueda exhaustiva sobre el hiperparámetro `alpha` de la regresión Ridge, definiendo y entrenando múltiples modelos para, finalmente, elegir el valor del hiperparámetro que resulte en el modelo con el mejor desempeño.

Primero crearemos un objeto de la clase `Ridge()`, que puede recibir como parámetro el valor de `alpha`. Este hiperparámetro corresponde a la constante que acompaña al término de penalización, es decir, `alpha` es usado para el control de la complejidad. En este caso no es necesario especificarlo, ya que cambiaremos su valor en cada iteración de la búsqueda:

In [ ]:
ridge = Ridge()

Ahora vamos a definir un objeto de la clase `KFold()`, que toma el conjunto de entrenamiento original y lo separa en k grupos, usando uno como validación y el resto (k-1) como entrenamiento. En este caso, definiremos `k=10` y usaremos el parámetro `shuffle=True` para indicar que se cambie el orden de los datos antes de separarlos en los grupos:

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state = 0)

Finalmente utilizaremos un diccionario para definir nuestro espacio de búsqueda de hiperparámetros, es decir, los valores que vamos a probar y sobre los que decidiremos cuál escoger. Almacenaremos estos valores en la variable `param_grid`:

In [ ]:
param_grid = {'alpha': [1, 2, 5, 10, 15, 20, 100, 500]}

A continuación, vamos a utilizar `GridSearchCV` para realizar la búsqueda exhaustiva del mejor hiperparámetro. Esta clase nos permitirá entrenar un modelo con todos los conjuntos obtenidos mediante el objeto `kfold`, además de todos los valores específicos para el hiperparámetro, seleccionando el mejor modelo entre todas las combinaciones posibles. En ese orden de ideas, definiremos el algoritmo `ridge`, los valores del hiperparámetro `param_grid`, y la estrategia de validación cruzada `kfold`: 

* La complejidad del modelo afecta el tiempo que tarda en entrenar, por lo que esta parte puede tardar más tiempo en completarse, puedes intentar agregar el parámetro `n_jobs=-1` para mejorar el rendimiento o usar `n_jobs=1` para limitarlo en caso de que el entorno falle

In [ ]:
modelos_grid = GridSearchCV(ridge, param_grid, cv=kfold, n_jobs=-1)

Finalmente, entrenaremos los modelos con los conjuntos definidos previamente:

* la línea `%%time` permite ver cuánto tiempo tardó en ejecutarse

In [ ]:
%%time
modelos_grid.fit(x_train, y_train)

Obtendremos el mejor parámetro usando el atributo `best_params_`:

In [ ]:
print("Mejor parámetro: {}".format(modelos_grid.best_params_)) 

Como puedes ver, el mejor valor del hiperparámetro `alpha` es 1. Posteriormente, almacenaremos el modelo ya entrenado usando el atributo `best_estimator_`

In [ ]:
mejor_modelo = modelos_grid.best_estimator_

Finalmente, obtendremos los coeficientes del modelo de regresión regularizada utilizando el atributo `coef_`:

In [ ]:
pd.DataFrame(zip(x_train.columns, mejor_modelo.coef_),columns=["Variable","Coeficiente"])

Una de las particularidades de la regresión Ridge es que, generalmente, los coeficientes resultantes tienen una magnitud menor a los coeficientes obtenidos con regresión lineal simple. Como puedes observar en nuestro mejor modelo, algunos de los coeficientes disminuyen. Debes tener en cuenta que la reducción de coeficientes es dependiente del valor del hiperparámetro `alpha`, por lo que en este caso la regularización no es tan fuerte y la reducción no es tan evidente.

### Evaluación del mejor modelo

A continuación realizaremos predicciones sobre el conjunto de pruebas para comparar con los valores de `y_test`. Utilizaremos la función `predict()` sobre el mejor modelo:

In [ ]:
y_pred = mejor_modelo.predict(x_test)

print('------ Modelo de regresión Ridge----')
print("RMSE: %.2f" % root_mean_squared_error(y_test, y_pred))
print("MAE: %.2f" % mean_absolute_error(y_test, y_pred))
print('R²: %.2f' % r2_score(y_test, y_pred))

Para este conjunto de datos, el valor de R<sup>2</sup> se mantiene en 0.71 y, adicionalmente, el RMSE y el MAE mantienen valores similares al modelo de regresión lineal. Es decir, según las trés métricas, el rendimiento de generalización no es mejor con regularización. ¿El rendimiento podría mejorar si creas un modelo mezclando regresión polinomial con regularización?

## Cierre

En este tutorial hemos utilizado nuevas clases de scikit-learn para entrenar modelos de regresión Ridge. Adicionalmente, observamos cómo escalar datos, definimos un objeto para realizar validación cruzada con k-Folds y realizamos una búsqueda exhaustiva de hiperparámetros. 

---
Si quieres más información sobre regresión Ridge en `scikit_learn` puedes consultar el [sitio web oficial](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)

Para la estandarización de datos con la clase `MinMaxScaler()` puedes consultar [este enlace](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

En particular, puedes ver el efecto que tiene cada escalador de Scikit-learn en [este ejemplo de scikit-learn](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#)

Para la búsqueda de hiperparámetros con la clase `GridSearchCV()` puedes consultar [este enlace](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

Finalmente, para obtener más información sobre la clase `KFold()` puedes ir [aquí](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)

---
*Creado por: Nicolás Díaz*

*Última edición: Camilo Rozo*

*Revisado por: Haydemar Nuñez*

*Versión: Enero 2025*  

*Universidad de los Andes*  